Basic Machine Learning Project

Authors: Oscar Gamarra; Manuel Ayllon

In [ ]:
# Write down importations and functions you need
import csv

In [1]:
# Load the data:
df_train = open('train_2024.csv')
df_test = open('test_2024.csv')

**EXERCISE 1**

Do some exploratory data analysis. Don’t just look globally, also look at (some of) these metrics by class. There might be differences, after all:  
- How many missing data are there?  
- Are there other strange values in the data?  
- What is the proportion of classes in your data?  
- How many genes (features) have entirely equal measured expression values as other genes (i.e. are there duplicate columns in the data)?  
- What is the maximum value in the data, and the minimum?  
- Which feature has the highest variance?  
- Finally, make a boxplot of the expression values of the 30 features with the highest 
variance in the dataset, ordered by this variance.